In [28]:
import base64
import requests
from urllib.parse import urlencode
import datetime
client_id = '253dcf58536d496fbcc51cf1c5c81831'
client_secret = '1d7148df4a0c42bab111637c6519f28d'
client_creds = f'{client_id}:{client_secret}'
client_creds_b64 = base64.b64encode(client_creds.encode())

In [88]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        } 
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            raise Exception("Could not authenticate client.")
            # return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token() 
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
        
        
    def get_resource(self, lookup_id, resource_type='albums', version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    def get_album(self, _id):
        return self.get_resource(_id, resource_type='albums')
    
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type='artists')
    
    def base_search(self, query_params): # type
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        lookup_url = f"{endpoint}?{query_params}"
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):  
            return {}
        return r.json()
    
    def search(self, query=None, operator=None, operator_query=None, search_type='artist' ):
        if query == None:
            raise Exception("A query is required")
        if isinstance(query, dict):
            query = " ".join([f"{k}:{v}" for k,v in query.items()])
        if operator != None and operator_query != None:
            if operator.lower() == "or" or operator.lower() == "not":
                operator = operator.upper()
                if isinstance(operator_query, str):
                    query = f"{query} {operator} {operator_query}"
        query_params = urlencode({"q": query, "type": search_type.lower()})
        print(query_params)
        return self.base_search(query_params)
    def get_playlists(self):
        endpoint = f"https://api.spotify.com/v1/playlists/4ziiwwWdBbxplDP55bEnOv/tracks"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return r.status_code
        return r.json()
    def get_audio_features(self):
        endpoint = f"https://api.spotify.com/v1/audio-features/5XeFesFbtLpXzIVDNQP22n,5oYRr51VatOtkFeEOursuZ,0LtOwyZoSNZKJWHqjzADpW,4FE9SgxcOoCNuGnaaU8TXz,22xXzpcl3VPGyRy8xYdBhB,4Psh3fEnAMftNPOTsAHPgG,3FTRSa9mWB3kKx2jkhUAN3,6fFTPGGItuEpjwXRUIr4rk,6ZmtxXUXRVjxOhugKkmerC,1jjSjma0cfNUhyOUzQqt6J,26u6HGjzExhRZD5ifgTBnG,2y7JqO7Jc7ErdOmxpvifRF,6JYk44BcllTsPVvZ8Rp6ZB,5iK8GmW2uiPuUc5CtkZrIK,7ri8Tn6Uq4yA9LjpI5dje0,4jcGmCcMyiGpgUHgSIKcc1,1aGpc5gn7HxnPHgGRvobqX,5Wr9Fl0NEWAkKIUB9ibOSV,6K4t31amVTZDgR3sKmwUJJ,6ypzSoz38iTjCDIqyLpEbT,46eK1kembooWblMi7M7yLM,3vgiIJW3iSQRRvgYiW86i9,6qMB7GsJd7L8sqEvQGWQl6,6YycppSAeGtSvwmu8qCcr2,3K80odWx5PcN615lno80L7,1cVpUMiPA6xYG1XK3N334Z,6RtPijgfPKROxEzTHNRiDp,64RkEV5b7bG697vhIPMlrl,7CydprSkKHANXosbr23hnR,5K9AcgSi8DK4lZKM2LpFu8,0VxocBntP1XTZRsR9ZURPS,2RSHsoi04658QL5xgQVov3,6HguG9HRb1Ke1bhihfE4m8,26OEFQnUYwPWmCoxZ2ox5Y,5xTEkzHycz81QTcqCS9oVv,1BlQWQgGP84r4GYUVty4Ar,0PyGOEylGJ63YvEtA7Dw9P,3gJxSEQtZ08xpRuAMdynHW,5YpzByJxStqpvsz9TXnONA,7laJx5tDtvm3Mkj832oxSb,5w1ingzqrsJka9nlnEFB64,6C2WXdMMndPrnhSJJolPu8,4DE42oDol0KHxympBsaiYu,2LskIZrCeLxRvCiGP8gxlh,6d2MCbByfqhU85Fmj8WCAN,7pBsquIkbED6W6uSQJGbkn,2x2x2U3sAv6IP7O0F9zEvc,669PEr8I3wOswY8BANBpdh,7EDsUT0289qe2W6N2vTe47,2Q5902VVPT2dePViBXULjI,5FXMRdJjKq1BIX4e8Eg9mK,6nZiYSBwPQ7fYnVWkkkj4g,20PiXVXC1skCs9vfyMxnKe,6Tvzf3VEi16JMhAgOwdt2y,3sZOt6h6Qq9aqtLbE6UMG7,47Bg6IrMed1GPbxRgwH2aC,1iQDltZqI7BXnHrFy4Qo1k,583f5m5Y74xBEd6kPAv7Q8,6U0FIYXCQ3TGrk4tFpLrEA,1C9DajYq19423BwN85bNf0,3hkC9EHFZNQPXrtl8WPHnX,7n6t89yx1U566KfgFbZNBD,1PSBzsahR2AKwLJgx8ehBj,40YcuQysJ0KlGQTeGUosTC,3GCdLUSnKSMJhs4Tj6CV3s,4YkN3JlceD5n2aMqL1kFnH,2t0Ci12vaJaYzEmsJL2S1X"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return r.status_code
        return r.json()
        

In [89]:
spotify = SpotifyAPI(client_id, client_secret)

In [90]:
spotify.perform_auth()

True

In [91]:
spotify.access_token

'BQBrESJwj0eAycPekWCuQvxoEsAoB5e33Lh0HO_VA1uxr_thf9Tg0TglODHcG8YE5hoDRoCsYWgD-uHG3WU'

In [92]:
playlist_dict = spotify.get_playlists()

In [93]:
print(playlist_dict)

{'href': 'https://api.spotify.com/v1/playlists/4ziiwwWdBbxplDP55bEnOv/tracks?offset=0&limit=100', 'items': [{'added_at': '2019-06-06T17:55:52Z', 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/fd1ewcnir8wm04lucbwdg6fnc'}, 'href': 'https://api.spotify.com/v1/users/fd1ewcnir8wm04lucbwdg6fnc', 'id': 'fd1ewcnir8wm04lucbwdg6fnc', 'type': 'user', 'uri': 'spotify:user:fd1ewcnir8wm04lucbwdg6fnc'}, 'is_local': False, 'primary_color': None, 'track': {'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7Ln80lUS6He07XvHI8qqHH'}, 'href': 'https://api.spotify.com/v1/artists/7Ln80lUS6He07XvHI8qqHH', 'id': '7Ln80lUS6He07XvHI8qqHH', 'name': 'Arctic Monkeys', 'type': 'artist', 'uri': 'spotify:artist:7Ln80lUS6He07XvHI8qqHH'}], 'available_markets': ['AD', 'AE', 'AR', 'AT', 'BE', 'BG', 'BH', 'BO', 'BR', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DE', 'DK', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FR', 'GB', 'GR', 'GT', 'HK',

In [94]:
playlist_dict['items'][0]['track']['id']

'5XeFesFbtLpXzIVDNQP22n'

In [95]:
for i in range(len(playlist_dict['items'])):
    print(playlist_dict['items'][i]['track']['id'], end=',')

5XeFesFbtLpXzIVDNQP22n,5oYRr51VatOtkFeEOursuZ,0LtOwyZoSNZKJWHqjzADpW,4FE9SgxcOoCNuGnaaU8TXz,22xXzpcl3VPGyRy8xYdBhB,4Psh3fEnAMftNPOTsAHPgG,3FTRSa9mWB3kKx2jkhUAN3,6fFTPGGItuEpjwXRUIr4rk,6ZmtxXUXRVjxOhugKkmerC,1jjSjma0cfNUhyOUzQqt6J,26u6HGjzExhRZD5ifgTBnG,2y7JqO7Jc7ErdOmxpvifRF,6JYk44BcllTsPVvZ8Rp6ZB,5iK8GmW2uiPuUc5CtkZrIK,7ri8Tn6Uq4yA9LjpI5dje0,4jcGmCcMyiGpgUHgSIKcc1,1aGpc5gn7HxnPHgGRvobqX,5Wr9Fl0NEWAkKIUB9ibOSV,6K4t31amVTZDgR3sKmwUJJ,6ypzSoz38iTjCDIqyLpEbT,46eK1kembooWblMi7M7yLM,3vgiIJW3iSQRRvgYiW86i9,6qMB7GsJd7L8sqEvQGWQl6,6YycppSAeGtSvwmu8qCcr2,3K80odWx5PcN615lno80L7,1cVpUMiPA6xYG1XK3N334Z,6RtPijgfPKROxEzTHNRiDp,64RkEV5b7bG697vhIPMlrl,7CydprSkKHANXosbr23hnR,5K9AcgSi8DK4lZKM2LpFu8,0VxocBntP1XTZRsR9ZURPS,2RSHsoi04658QL5xgQVov3,6HguG9HRb1Ke1bhihfE4m8,26OEFQnUYwPWmCoxZ2ox5Y,5xTEkzHycz81QTcqCS9oVv,1BlQWQgGP84r4GYUVty4Ar,0PyGOEylGJ63YvEtA7Dw9P,3gJxSEQtZ08xpRuAMdynHW,5YpzByJxStqpvsz9TXnONA,7laJx5tDtvm3Mkj832oxSb,5w1ingzqrsJka9nlnEFB64,6C2WXdMMndPrnhSJJolPu8,4DE42oDol0KHxympBsaiYu,2LskIZrCeLx

In [96]:
spotify.get_audio_features()

400